In [6]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('corpus.txt').read()))



In [7]:
#print(WORDS)

In [8]:
def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [9]:
#led = edits2('weke')


In [10]:
with open('input.txt', 'r') as reader, open('out.txt','w') as writer:
    lines_out = []
    for line_count, line in enumerate(reader):
        lines_out.append(line)
        words = re.findall(r'\w+', line)
        for word_count, word in enumerate(words):
            if re.fullmatch(r'\d+', word):
                continue
            corr_word = correction(word.lower())
            if(corr_word != word.lower()):
                # find typo
                if(word[0].isupper()):
                    corr_word = corr_word.title()
                lines_out[-1] = re.sub(word, corr_word, lines_out[-1])
                print(f'{word} in line {line_count+1}, word {word_count+1}, replaced by {corr_word}.')
    writer.writelines(lines_out)

loue in line 1, word 2, replaced by love.
ybu in line 1, word 3, replaced by you.
Smih in line 1, word 5, replaced by Smith.
nane in line 1, word 8, replaced by name.
Quebuc in line 1, word 10, replaced by Quebec.
